In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from sklearn.metrics import mean_squared_error
from datetime import datetime
plt.style.use('fivethirtyeight')

In [ ]:
# Fetch Bitcoin historical data from Yahoo Finance
ticker = "BTC-USD"
end=datetime.now() # current date
start = datetime(end.year-15,end.month,end.day) # 15 years before the current date 
data = yf.download(ticker, start=start, end=end)

In [ ]:
data

In [ ]:
data = data[['Close']]
data = data.dropna()

In [ ]:
data

In [ ]:
# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

def create_sequences(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [ ]:
# Split the data into training and testing sets
time_step = 60
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

X_train, y_train = create_sequences(train_data, time_step)
X_test, y_test = create_sequences(test_data, time_step)

In [ ]:
# Reshape data for LSTM/GRU
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Function to build models
def build_model(model_type='LSTM'):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(50, return_sequences=False))
    elif model_type == 'GRU':
        model.add(GRU(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(Dropout(0.2))
        model.add(GRU(50, return_sequences=False))
    
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Train and evaluate LSTM model
lstm_model = build_model('LSTM')
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Train and evaluate GRU model
gru_model = build_model('GRU')
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Function to predict future prices
def predict_future(model, last_data, future_days=30):
    future_predictions = []
    input_seq = last_data[-time_step:].reshape(1, time_step, 1)
    
    for _ in range(future_days):
        pred = model.predict(input_seq)[0][0]
        future_predictions.append(pred)
        new_input = np.append(input_seq[:, 1:, :], [[[pred]]], axis=1)
        input_seq = new_input
    
    return scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

In [ ]:
# Predict future prices
future_days = 30
y_future_lstm = predict_future(lstm_model, data_scaled, future_days)
y_future_gru = predict_future(gru_model, data_scaled, future_days)

In [ ]:
# Predictions for test set
y_pred_lstm = lstm_model.predict(X_test)
y_pred_gru = gru_model.predict(X_test)

In [ ]:
y_pred_lstm = scaler.inverse_transform(y_pred_lstm.reshape(-1, 1))
y_pred_gru = scaler.inverse_transform(y_pred_gru.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Calculate RMSE
rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
rmse_gru = np.sqrt(mean_squared_error(y_test, y_pred_gru))

print(f"LSTM RMSE: {rmse_lstm}")
print(f"GRU RMSE: {rmse_gru}")

In [ ]:
# Plot results
plt.figure(figsize=(12,6))
plt.plot(y_test, label='Actual Price', color='black')
plt.plot(y_pred_lstm, label='LSTM Predicted Price', color='blue')
plt.plot(y_pred_gru, label='GRU Predicted Price', color='red')
plt.legend()
plt.show()

In [ ]:
# Plot future predictions
plt.figure(figsize=(12,6))
plt.plot(range(len(y_future_lstm)), y_future_lstm, label='LSTM Future Predictions', color='blue')
plt.plot(range(len(y_future_gru)), y_future_gru, label='GRU Future Predictions', color='red')
plt.legend()
plt.title("Future Price Predictions")
plt.show()